### Check the active and reactive power loss in the data
- Useful to compute active and reactive power loss of the solvers

In [1]:
import torch
import pandas as pd
from gridFM.datasets.powergrid import GridDatasetMem
from gridFM.datasets.data_normalization import *
from torch_geometric.loader import DataLoader
from tqdm import tqdm
from gridFM.utils.loss import *
from gridFM.io.param_handler import *
from gridFM.datasets.utils import split_dataset

torch.set_printoptions(precision=10)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
data_path1 = r"C:\Users\MatteoMazzonelli\Desktop\gridfm_model_evaluation\scripts\..\data\case300_ieee_dc"
scenarios = 512
norm_method = "baseMVAnorm"
node_normalizer, edge_normalizer = BaseMVANormalizer(True), BaseMVANormalizer(False) 
mask_ratio = 0.5
mask_dim = 6
mask_value = 0

In [3]:
dataset = GridDatasetMem(
        root=data_path1,
        norm_method=norm_method,
        node_normalizer=node_normalizer,
        edge_normalizer=edge_normalizer,
        mask_ratio=mask_ratio,
        mask_dim=mask_dim,
    )

In [4]:
train_dataset, val_dataset, test_dataset = split_dataset(
                dataset, data_path1, 0.1, 0.1
            )

In [5]:
loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [6]:
loss = PBELoss()

In [7]:
loss_sum = 0.0
power_loss_sum = 0.0
active_power_loss_sum = 0.0
reactive_power_loss_sum = 0.0
num_batches = 0

# Loop through the batches
for batch in tqdm(loader):
    metric = loss(batch.y, batch.y, batch.edge_index, batch.edge_attr, batch.mask)
    
    # Accumulate the metrics
    loss_sum += metric['loss'].item()
    power_loss_sum += metric['Power power loss in p.u.']
    active_power_loss_sum += metric['Active Power Loss in p.u.']
    reactive_power_loss_sum += metric['Reactive Power Loss in p.u.']
    num_batches += 1

# Calculate the means
mean_loss = loss_sum / num_batches
mean_power_loss = power_loss_sum / num_batches
mean_active_power_loss = active_power_loss_sum / num_batches
mean_reactive_power_loss = reactive_power_loss_sum / num_batches

# Print the results

    

100%|██████████| 6/6 [00:03<00:00,  1.51it/s]


In [8]:
print("BaseMVA: ", node_normalizer.baseMVA)

BaseMVA:  tensor(2465.)


In [9]:
print(f"Mean Power Power Loss: {mean_power_loss*node_normalizer.baseMVA}")
print(f"Mean Active Power Loss in MW: {mean_active_power_loss*node_normalizer.baseMVA}")
print(f"Mean Reactive Power Loss in MVar: {mean_reactive_power_loss*node_normalizer.baseMVA}")

Mean Power Power Loss: 70.902587890625
Mean Active Power Loss in MW: 6.602354049682617
Mean Reactive Power Loss in MVar: 69.76075744628906
